In [78]:
import sys
sys.path.append("../SearchingOptimalPruningAmount/")

In [79]:
from models import MVCNN
from tools import ImgDataset
import torch

In [80]:
model = MVCNN.SVCNN(torch.load('model-00030.pth', map_location='mps'))

In [81]:
test_dataset = ImgDataset.SingleImgDataset(
    root_dir='../SearchingOptimalPruningAmount/ModelNet40-12View/*/test',
    scale_aug=False,
    rot_aug=False,
    test_mode=True,
    num_models=1000,
    num_views=12
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [82]:
paths_f = sorted(test_loader.dataset.filepaths, key=lambda x: int(x.split('/')[-1].split('_')[-1].split('.')[0]))

In [83]:
len(paths_f)

22320

In [92]:
view_0, view_1, view_2, view_3, view_4, view_5, view_6, view_7, view_8, view_9, view_10, view_11 = [], [], [], [], [], [], [], [], [], [], [], []

for i in range(len(paths_f)):
    view_i = paths_f[i].split('/')[-1].split('_')[-1].split('.')[0]
    if view_i == '0':
        view_0.append(paths_f[i])
    elif view_i == '1':
        view_1.append(paths_f[i])
    elif view_i == '2':
        view_2.append(paths_f[i])
    elif view_i == '3':
        view_3.append(paths_f[i])
    elif view_i == '4':
        view_4.append(paths_f[i])
    elif view_i == '5':
        view_5.append(paths_f[i])
    elif view_i == '6':
        view_6.append(paths_f[i])
    elif view_i == '7':
        view_7.append(paths_f[i])
    elif view_i == '8':
        view_8.append(paths_f[i])
    elif view_i == '9':
        view_9.append(paths_f[i])
    elif view_i == '10':
        view_10.append(paths_f[i])
    elif view_i == '11':
        view_11.append(paths_f[i])
print(len(view_0), len(view_1), len(view_2), len(view_3), len(view_4), len(view_5), len(view_6), len(view_7), len(view_8), len(view_9), len(view_10), len(view_11))

1860 1860 1860 1860 1860 1860 1860 1860 1860 1860 1860 1860


In [ ]:
svcnn = model.net_1
mvcnn = model.net_2

def pooler(svcnn_out):
    pooled_features, _ = torch.max(torch.stack(svcnn_out), dim=0, keepdim=False)